In [1]:
import os
import sys

# Importing utils file in a notebook requires to add the directory in the 
# sys path before importing it.
utilsModulePath = os.path.abspath("../utils")
if (utilsModulePath not in sys.path):
    sys.path.append(utilsModulePath)
# Now we can import utils
import utils

# Blood Alcohol domain

Available at: <https://github.com/gateslm/Blood-Alcohol-Domain>

```math
@misc{BAC23,
  title = {Blood Alcohol Content Domain},
  howpublished={\url{https://github.com/gateslm/Blood-Alcohol-Domain}}, 
  year = {2023},
  url={https://github.com/gateslm/Blood-Alcohol-Domain}, 
  author = {Doyle, D{\'o}nal and Cunningham, P{\'a}draig, and Coyle, Lorcan}
}
```

Used in: <https://link-springer-com.bucm.idm.oclc.org/chapter/10.1007/978-3-031-40177-0_10>

In [41]:
import json
# Opening JSON file
with open('./blood-alcohol-domain.json') as f:
    dataFile = json.load(f)
data = dataFile["cases"]

In [42]:
object_data = { case["casename"]: case for case in data }
object_data

{'n1': {'casename': 'n1',
  'Gender': 'male',
  'FrameSize': 1,
  'AmountConsumed': 1,
  'Meal': 'snack',
  'Duration': 60,
  'solution': '0.2'},
 'n2': {'casename': 'n2',
  'Gender': 'female',
  'FrameSize': 2,
  'AmountConsumed': 3,
  'Meal': 'none',
  'Duration': 120,
  'solution': '0.8'},
 'n3': {'casename': 'n3',
  'Gender': 'female',
  'FrameSize': 4,
  'AmountConsumed': 4,
  'Meal': 'full',
  'Duration': 90,
  'solution': '0.8'},
 'n4': {'casename': 'n4',
  'Gender': 'male',
  'FrameSize': 4,
  'AmountConsumed': 6,
  'Meal': 'none',
  'Duration': 120,
  'solution': '1.0++'},
 'n5': {'casename': 'n5',
  'Gender': 'male',
  'FrameSize': 4,
  'AmountConsumed': 3,
  'Meal': 'none',
  'Duration': 60,
  'solution': '0.5'},
 'n6': {'casename': 'n6',
  'Gender': 'male',
  'FrameSize': 5,
  'AmountConsumed': 4,
  'Meal': 'none',
  'Duration': 60,
  'solution': '0.6'},
 'n7': {'casename': 'n7',
  'Gender': 'male',
  'FrameSize': 5,
  'AmountConsumed': 7,
  'Meal': 'none',
  'Duration': 12

In [43]:
import cbrkit
casebase= object_data

## Gender, Amount consumed, meal and duration

In [44]:
def nominal_range_similarity(listNominalValues: list):
    """Similarity function for a range of nominal values
    It works like the range similarity function but using 
    value indexes
    """
    min = 0
    max = len(listNominalValues)-1
    range = max
    def wrapped_function(x:str, y:str):
        if (x in listNominalValues)  and (y in listNominalValues):
            ix = listNominalValues.index(x)
            iy = listNominalValues.index(y)
            normalizeX = (ix - min) / range
            normalizeY = (iy - min) / range
            return 1-abs(normalizeX - normalizeY)
        else:
            return 0.0
    return wrapped_function

In [45]:
simFunction = cbrkit.sim.attribute_value(
    attributes= {
        "Gender": cbrkit.sim.generic.equality(),
        "AmountConsumed": cbrkit.sim.numbers.linear_interval(1,14),
        "Meal": nominal_range_similarity (['none', 'snack', 'full']),
        "Duration": cbrkit.sim.numbers.linear_interval(30,240)
    },
    aggregator=cbrkit.sim.aggregator(pooling="mean")
)

In [46]:
#similarityData = utils.compute_similarity_data(casebase, simFunction, idAttribute="casename")
retriever = cbrkit.retrieval.build(
        simFunction
    )
result = cbrkit.retrieval.apply_queries(casebase, casebase, retriever)

In [47]:
from cbrkit.model import Result, ResultStep
def getSteps(self):
    return self.steps
def getAllSimilarityData(self):
    data = {}
    for query in self.queries.keys():
        data[query] = self.queries[query].similarities
    return data
def getQValues(self):
    return self.queries.keys()

def getSimilaritiesQ(self, Q):
    return self.queries[Q].similarities

Result.getSteps = getSteps
ResultStep.getAllSimilarityData = getAllSimilarityData
ResultStep.getQValues = getQValues
ResultStep.getSimilaritiesQ = getSimilaritiesQ

steps = result.getSteps()
data = steps[0].getAllSimilarityData()

In [51]:
import orjson

def default(obj):
    if isinstance(obj, np.float64):
        return float(obj)
    raise TypeError

simprint = orjson.dumps(dict(similarityConfiguration=result.metadata,similarityScores=data), default=default, option=orjson.OPT_NON_STR_KEYS)
file_path = "cbrkit_GenderAmountMealDuration.json"
with open(file_path, "w") as file:
    file.write(simprint.decode("utf-8"))

In [30]:
cbrkit.dumpers.file("casebase.json",casebase)

In [ ]:
# Wrap with similarity description
similarityDataComplete = {
    "similarityConfigurationon": {
    "globalSim": {
      "simFunction": "Weighted average"
    },
    "localSim": {
      "Gender": {
        "simFunction": "equals",
        "weight": 0.25,
        "description": "String equality"
      },
      "AmountConsumed": {
        "simFunction": "RangeSimilarity",
        "weight": 0.25,
        "description": "Similarity normalized in range (1,14)"
      },
      "Meal": {
        "simFunction": "NominalRangeSimilarity",
        "weight": 0.25,
        "description": "Similarity in an ordered cualitative range (none, snack, full)"
      },
      "Duration": {
        "simFunction": "RangeSimilarity",
        "weight": 0.25,
        "description": "Similarity normalized in range (30,240)"
      }
    }
  }
}
similarityDataComplete["similarityScores"] = similarityData

In [7]:
utils.exportToJson(similarityDataComplete, "GenderAmountMealDuration.json")

## Gender, Amount consumed, meal and frame size

In [8]:

simFunction = cbrkit.sim.attribute_value(
    attributes= {
        "Gender": cbrkit.sim.generic.equality(),
        "AmountConsumed": cbrkit.sim.numbers.linear_interval(1,14),
        "Meal": utils.nominal_range_similarity (['none', 'snack', 'full']),
        "FrameSize": cbrkit.sim.numbers.linear_interval(1,8)
    },
    aggregator=cbrkit.sim.aggregator(pooling="mean")
)

In [9]:
similarityData = utils.compute_similarity_data(casebase, simFunction, idAttribute="casename")

In [ ]:
# Wrap with similarity description
similarityDataComplete = {
    "similarityConfigurationon": {
    "globalSim": {
      "simFunction": "Weighted average"
    },
    "localSim": {
      "Gender": {
        "simFunction": "equals",
        "weight": 0.25,
        "description": "String equality"
      },
      "AmountConsumed": {
        "simFunction": "RangeSimilarity",
        "weight": 0.25,
        "description": "Similarity normalized in range (1,14)"
      },
      "Meal": {
        "simFunction": "NominalRangeSimilarity",
        "weight": 0.25,
        "description": "Similarity in an ordered cualitative range (none, snack, full)"
      },
      "FrameSize": {
        "simFunction": "RangeSimilarity",
        "weight": 0.25,
        "description": "Similarity normalized in range (1,8)"
      }
    }
  }
}
similarityDataComplete["similarityScores"] = similarityData

In [11]:
utils.exportToJson(similarityDataComplete, "GenderAmountMealFrameSize.json")

## Amount consumed, meal and frame size

In [12]:
simFunction = cbrkit.sim.attribute_value(
    attributes= {
        "AmountConsumed": cbrkit.sim.numbers.linear_interval(1,14),
        "Meal": utils.nominal_range_similarity (['none', 'snack', 'full']),
        "FrameSize": cbrkit.sim.numbers.linear_interval(1,8)
    },
    aggregator=cbrkit.sim.aggregator(pooling="mean")
)

In [13]:
similarityData = utils.compute_similarity_data(casebase, simFunction, idAttribute="casename")

In [ ]:
# Wrap with similarity description
similarityDataComplete = {
    "similarityConfigurationon": {
    "globalSim": {
      "simFunction": "Weighted average"
    },
    "localSim": {
      "AmountConsumed": {
        "simFunction": "RangeSimilarity",
        "weight": 0.25,
        "description": "Similarity normalized in range (1,14)"
      },
      "Meal": {
        "simFunction": "NominalRangeSimilarity",
        "weight": 0.25,
        "description": "Similarity in an ordered cualitative range (none, snack, full)"
      },
      "FrameSize": {
        "simFunction": "RangeSimilarity",
        "weight": 0.25,
        "description": "Similarity normalized in range (1,8)"
      }
    }
  }
}
similarityDataComplete["similarityScores"] = similarityData

In [15]:
utils.exportToJson(similarityDataComplete, "AmountMealFrameSize.json")